In [17]:
import os

from bs4 import BeautifulSoup

In [36]:
for folder_name in os.listdir('data/allrecipes/'):
    folder_path = 'data/allrecipes/' + folder_name
    for file_name in os.listdir(folder_path):
        file_path = folder_path + '/' + file_name
        with open(file_path) as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
            info = soup.find('recipe-signup')
            print(info['data-id'])
            print(info['data-title'])
            ingredients = soup.find_all('span', 'recipe-ingred_txt', 'itemprop=ingredient')
            for ingredient in ingredients:
                if 'itemprop' in ingredient.attrs and ingredient['itemprop'] == 'ingredients':
                    print(ingredient.string)
            print()

6902
"Indian Naan I"

            1 1/2 teaspoons active dry yeast
           

            2 cups bread flour
           

            1 teaspoon salt
           

            9 tablespoons water
           

            2 tablespoons clarified butter
           

6903
"Grandma\u0027s English Muffin Bread"

            3 cups all-purpose flour
           

            2 1/4 teaspoons active dry yeast
           

            1/2 tablespoon white sugar
           

            1 teaspoon salt
           

            1/8 teaspoon baking powder
           

            1 cup warm milk
           

            1/4 cup water
           

6905
"Apple Scones"

            2 cups all-purpose flour
           

            1/4 cup white sugar
           

            2 teaspoons baking powder
           

            1/2 teaspoon baking soda
           

            1/2 teaspoon salt
           

            1/4 cup butter, chilled
           

            1 apple - peeled, cored and shredded

In [4]:
total_urls  = 0
total_recipes = 0
recipe_errors = 0
response_errors = 0

base_url = 'http://allrecipes.com/recipe/'
lower_limit = 6663
upper_limit = 246200
# for ide in range(lower_limit, upper_limit + 1):
for ide in range(6901, 6906):
    try:
        str_id = str(ide)
        folder_path = 'data/allrecipes/' + str(ide // 1000 * 1000)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        file_path = folder_path + '/' + str_id + '.html'
        if not os.path.isfile(file_path):
            url = base_url + str_id
            response = requests.get(url)
            if response.ok:
                soup = BeautifulSoup(response.content, 'html.parser')
                if is_a_valid_recipe(soup):
                    with open(file_path, 'w') as f:
                        f.write(soup.prettify())
                        total_recipes += 1
                else:
                    recipe_errors += 1
            else:
                response_errors += 1
    except:
        pass
    total_urls += 1
    if total_urls % 1000 == 0:
        print(total_urls, 'processed.')

print('Total urls:', total_urls)
print('Total recipes:', total_recipes)
print('Recipe errors:', recipe_errors)
print('Response errors:', response_errors)

Total urls: 5
Total recipes: 1
Recipe errors: 0
Response errors: 0


In [15]:
url = 'http://allrecipes.com/recipe/139620'
url = 'http://allrecipes.com/recipe//8919'
r = requests.get(url, headers={'referer': 'http://allrecipes.com/recipes/231/world-cuisine/european/'})
soup = BeautifulSoup(r.content, 'html.parser')

In [16]:
with open('test.html', 'w') as f:
    f.write(soup.prettify())